# Predictions Sales

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.preprocessing import TargetEncoder
from category_encoders import TargetEncoder
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.pipeline import Pipeline


pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
def extract_color(series):
    """
    Extracts the color from a pandas Series of product descriptions,
    assuming that the color appears after an 8-digit number.
    
    Example:
      Input: 'Via Uno Botin 12231407 Marron'
      Output: 'Marron'
      
    After extraction, any missing values (NaN) are replaced by the most
    frequent color found in the Series.
    
    Parameters:
        series (pd.Series): A Series containing product descriptions.
        
    Returns:
        pd.Series: A Series with the extracted colors, with NaNs replaced by the mode.
    """
    # Apply the regex to extract the color; the lambda applies the search once per element.
    extracted = series.apply(
        lambda desc: (lambda m: m.group(1).strip() if m else None)(
            re.search(r'\d{8}\s+(.*)$', str(desc))
        )
    )
    
    # Compute the mode (most frequent value) of the extracted colors
    mode_color = extracted.mode()[0] if not extracted.mode().empty else None
    
    # Replace NaN values with the mode color
    return extracted.fillna(mode_color)
    
def extract_number_from_column(df, column_name):
    """
    Extracts numeric digits from the specified column, fills NaN values with the mode,
    and converts the result to integer type.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column to process.

    Returns:
        pd.DataFrame: The DataFrame with the processed column.
    """
    # Extract digits from the column (results in a Series)
    extracted = df[column_name].str.extract(r'(\d+)', expand=False)
    
    # Fill NaN values with the most frequent (mode) value
    mode_value = extracted.mode()[0] if not extracted.mode().empty else '0'
    extracted = extracted.fillna(mode_value)
    
    # Convert the extracted numbers to integer
    df[column_name] = extracted.astype(int)
    
    return df

def extract_date_parts(df, datetime_column='Fecha y Hora Venta'):
    """
    Extracts the month, day, and hour from a datetime column in the DataFrame,
    creating new columns 'Month', 'Day', and 'Hour'. Then, it drops the original datetime column.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame containing the datetime column.
        datetime_column (str): The name of the datetime column (default 'Fecha y Hora Venta').
    
    Returns:
        pd.DataFrame: The DataFrame with the new columns added and the original column removed.
    """
    df['Month'] = df[datetime_column].dt.month
    df['Day']   = df[datetime_column].dt.day
    df['Hour']  = df[datetime_column].dt.hour
    
    return df


In [3]:
# Load the data
data = pd.read_excel('/home/sebastian/Documents/Portfolio/Ventas via uno/EDA/Train.xlsx')
test_data = pd.read_excel("/home/sebastian/Documents/Portfolio/Ventas via uno/EDA/test.xlsx")
data.columns

Index(['Tipo Movimiento', 'Tipo Documento', 'Número Documento',
       'Fecha de Emisión', 'Tracking number', 'Fecha y Hora Venta', 'Sucursal',
       'Vendedor', 'Cliente Nombre', 'Cliente RUT', 'Cliente Email',
       'Cliente Dirección', 'Cliente Comuna', 'Cliente Ciudad',
       'Lista de precio', 'Tipo de entrega', 'Moneda',
       'Tipo de Producto/Servicio', 'SKU', 'Producto/Servicio', 'Variante',
       'Otros Atributos', 'Marca', 'Detalle de Productos/Servicios Pack/Promo',
       'Precio base', 'Precio Neto Unitario', 'Precio Bruto Unitario',
       'Cantidad', 'Subtotal Neto', 'Subtotal Impuestos', 'Subtotal Bruto',
       'Nombre de dcto', 'Descuento Neto', 'Descuento Bruto', '% Descuento',
       'Costo neto unitario', 'Costo Total Neto', 'Margen', '% Margen',
       'Producto', 'Marketplace'],
      dtype='object')

In [4]:
data.head()

Tipo Movimiento        Tipo Documento  Número Documento Fecha de Emisión  \
0           venta  boleta electrónica t             21923       2024-01-01   
1           venta  boleta electrónica t             21924       2024-01-01   
2           venta  boleta electrónica t             21925       2024-01-01   
3           venta  boleta electrónica t             21926       2024-01-01   
4           venta  boleta electrónica t             21927       2024-01-01   

            Tracking number  Fecha y Hora Venta     Sucursal      Vendedor  \
0  659208472266bd053b66938c 2023-12-31 21:33:10  Casa Matriz  VENTA ONLINE   
1  659213862266bd053e66957b 2023-12-31 22:21:10  Casa Matriz  VENTA ONLINE   
2  659239f57e379f0505d14b6e 2024-01-01 01:05:09  Casa Matriz  VENTA ONLINE   
3  659239f9cb05a8054ae57022 2024-01-01 01:05:13  Casa Matriz  VENTA ONLINE   
4  65923dd57e379f0505d14ba0 2024-01-01 01:21:41  Casa Matriz  VENTA ONLINE   

     Cliente Nombre Cliente RUT              Cliente Email Cliente Dirección  \
0   viviana salinas  13617340-5                  Sin datos         Sin datos   
1     Silvia Psijas   6693321-0                  Sin datos         Sin datos   
2  Priscilla Bustos  16753486-4                  Sin datos         Sin datos   
3  Priscilla Bustos  16753486-4                  Sin datos         Sin datos   
4  Carolina Montiel  15663772-6  montieliglesias@gmail.com           Caren     

         Cliente Comuna Cliente Ciudad        Lista de precio  \
0  region metropolitana       Santiago  Lista de Precios Base   
1  region metropolitana       Santiago  Lista de Precios Base   
2  region metropolitana       Santiago  Lista de Precios Base   
3  region metropolitana       Santiago  Lista de Precios Base   
4  region metropolitana       Santiago  Lista de Precios Base   

    Tipo de entrega Moneda Tipo de Producto/Servicio             SKU  \
0  retiro en tienda    CLP                     Botin  12231407015007   
1  retiro en tienda    CLP                     Botin  12231407023007   
2  retiro en tienda    CLP                  Sandalia  22323605015006   
3  retiro en tienda    CLP                  Sandalia  22323605015006   
4  retiro en tienda    CLP                      Bota  12231303015008   

                 Producto/Servicio  Variante  Otros Atributos  Marca  \
0    Via Uno Botin 12231407 Marron  37 Negro              NaN    NaN   
1    Via Uno Botin 12231407 Marron        37              NaN    NaN   
2  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
3  Via Uno Sandalia 22323605 Negro        36              NaN    NaN   
4      Via Uno Bota 12231303 Negro        38              NaN    NaN   

   Detalle de Productos/Servicios Pack/Promo  Precio base  \
0                                          0        10076   
1                                          0        10076   
2                                          0        12597   
3                                          0        12597   
4                                          0        10916   

   Precio Neto Unitario  Precio Bruto Unitario  Cantidad  Subtotal Neto  \
0                 10076                  11990         1          10076   
1                 10076                  11990         1          10076   
2                 12597                  14990         1          12597   
3                 12597                  14990         1          12597   
4                 10916                  12990         1          10916   

   Subtotal Impuestos  Subtotal Bruto Nombre de dcto  Descuento Neto  \
0                1914           11990      Sin datos               0   
1                1914           11990      Sin datos               0   
2                2393           14990      Sin datos               0   
3                2393           14990      Sin datos               0   
4                2074           12990      Sin datos               0   

   Descuento Bruto  % Descuento  Costo neto unitario  Costo Total Neto  \
0

In [5]:
data=data[data['Variante']!='Despacho']    
data=data[data['Variante']!='envio']
data=data[data['Tipo de Producto/Servicio']!='Sin Tipo']
data.columns

Index(['Tipo Movimiento', 'Tipo Documento', 'Número Documento',
       'Fecha de Emisión', 'Tracking number', 'Fecha y Hora Venta', 'Sucursal',
       'Vendedor', 'Cliente Nombre', 'Cliente RUT', 'Cliente Email',
       'Cliente Dirección', 'Cliente Comuna', 'Cliente Ciudad',
       'Lista de precio', 'Tipo de entrega', 'Moneda',
       'Tipo de Producto/Servicio', 'SKU', 'Producto/Servicio', 'Variante',
       'Otros Atributos', 'Marca', 'Detalle de Productos/Servicios Pack/Promo',
       'Precio base', 'Precio Neto Unitario', 'Precio Bruto Unitario',
       'Cantidad', 'Subtotal Neto', 'Subtotal Impuestos', 'Subtotal Bruto',
       'Nombre de dcto', 'Descuento Neto', 'Descuento Bruto', '% Descuento',
       'Costo neto unitario', 'Costo Total Neto', 'Margen', '% Margen',
       'Producto', 'Marketplace'],
      dtype='object')

In [6]:
# Verify null values
data.info()
display(data.isnull().sum())

# Drop columns with null values and columns that are not useful
data.drop(['Marca', 'Otros Atributos', 'Fecha de Emisión'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 27133 entries, 0 to 35437
Data columns (total 41 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   Tipo Movimiento                            27133 non-null  object        
 1   Tipo Documento                             27133 non-null  object        
 2   Número Documento                           27133 non-null  int64         
 3   Fecha de Emisión                           11654 non-null  datetime64[ns]
 4   Tracking number                            27133 non-null  object        
 5   Fecha y Hora Venta                         27133 non-null  datetime64[ns]
 6   Sucursal                                   27133 non-null  object        
 7   Vendedor                                   27133 non-null  object        
 8   Cliente Nombre                             27133 non-null  object        
 9   Cliente RUT           

Tipo Movimiento                                  0
Tipo Documento                                   0
Número Documento                                 0
Fecha de Emisión                             15479
Tracking number                                  0
Fecha y Hora Venta                               0
Sucursal                                         0
Vendedor                                         0
Cliente Nombre                                   0
Cliente RUT                                      0
Cliente Email                                    0
Cliente Dirección                                0
Cliente Comuna                                   0
Cliente Ciudad                                   0
Lista de precio                                  0
Tipo de entrega                                  0
Moneda                                           0
Tipo de Producto/Servicio                        0
SKU                                              0
Producto/Servicio              

# Features engineer

In [7]:
data['Cliente Ciudad'].unique()

array(['Santiago', 'Sin datos', 'REGIÓN METROPOLITANA', 'Viña Del Mar',
       'Región Metropolitana', 'STGO', 'santiago', 'CAÑETE', 'SANTIAGO',
       'Colina', 'Región del Libertador General Bernardo O’Higgins (',
       'antofagasta', 'RANCAGUA', 'Linares', 'Región de Valparaíso (V)',
       'Región Metropolitana de Santiago', 'Región de Los Lagos (X)',
       'Parral', 'Región de Antofagasta (II)', 'AYSEN', 'ARAUCO', 'LANCO',
       'OSORNO', 'TRAIGUEN', 'TOME', 'TALAGANTE', 'TALCAHUANO',
       'QUILLOTA', 'LOS ANDES', 'METROPOLITANA',
       "Región del Libertador General Bernardo O'Higgins",
       'Región del Biobío (VIII)', 'CALAMA', 'REGIÓN DE VALPARAÍSO (V)',
       'PUENTE ALTO', 'Copiapó', 'REGIÓN DEL ÑUBLE (XVI)', 'PUERTO MONTT',
       'Región de La Araucanía (IX)', 'FRUTILLAR', 'VALPARAISO',
       'Rancagua', 'Talca', 'xxx', 'Región de Tarapacá (I)',
       'METROPOLITANA DE SANTIAGO'], dtype=object)

In [8]:
# Split Fecha y Hora Venta in month, day and hour
data = extract_date_parts(data)

# Extract talla from variante
data = extract_number_from_column(data, 'Variante')
data['Variante'] = data['Variante'].astype(str)

data['Cliente Ciudad']= data['Cliente Ciudad'].str.lower()
data['Cliente Ciudad'] = data['Cliente Ciudad'].replace(['metropolitana','santiago'])
data['Cliente Ciudad']= data['Cliente Ciudad'].replace(['region metropolitana','santiago'])


In [9]:
# Extract color 
data['Color'] = extract_color(data['Producto/Servicio'])

data['Month_sin'] = np.sin(2 * np.pi * data['Month']/12)
data['Month_cos'] = np.cos(2 * np.pi * data['Month']/12)

# Predict volume of sales 
Feautures = ['Month_sin', 'Month_cos','Marketplace', 'Producto', 'Color', 'Precio Neto Unitario', 'Variante', 'Cliente Ciudad', '% Descuento']
Target=['Cantidad']

ventas = data[data['Tipo Movimiento']=='venta']
devolucion = data[data['Tipo Movimiento']=='devolucion']

devolucion['Cantidad']=devolucion['Cantidad']*-1

## Preprocessor

In [10]:
X = data[Feautures]
y = data[Target]

# Identify categorical columns
object_columns = X.select_dtypes(include=['object']).columns

# Preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('target_enc', TargetEncoder(), ['Marketplace', 'Producto', 'Color', 'Cliente Ciudad','Variante']),
        ('passthrough', 'passthrough', ['Month_sin', 'Month_cos', 'Precio Neto Unitario','% Descuento' ])
    ])

X_processed = preprocessor.fit_transform(X, y)

# Train Model

In [12]:
# Definir el modelo
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=12,
    min_samples_leaf=5,
    random_state=1
)

data = data.sort_values(by='Fecha y Hora Venta')
tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(X):
    # Obtener los datos sin procesar para cada split
    X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crear un nuevo preprocessor para este fold
    preprocessor_fold = ColumnTransformer(
        transformers=[
            ('target_enc', TargetEncoder(), ['Marketplace', 'Producto', 'Color', 'Cliente Ciudad', 'Variante']),
            ('passthrough', 'passthrough', ['Month_sin', 'Month_cos', 'Precio Neto Unitario', '% Descuento'])
        ])
    
    # Ajustar el preprocessor con los datos de entrenamiento y transformar ambos conjuntos
    X_train = preprocessor_fold.fit_transform(X_train_raw, y_train)
    X_test = preprocessor_fold.transform(X_test_raw)

    # Entrenar el modelo en el fold actual
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    
    # Evaluar el modelo
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE = {mse:.2f}, R² = {r2:.4f}")

MSE = 0.12, R² = 0.7617
MSE = 20.27, R² = 0.0899
MSE = 0.02, R² = 0.9535
MSE = 3.01, R² = -4.9353
MSE = 0.10, R² = 0.8449


# Stock recommend